In [34]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import datetime as dt
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [35]:
class NeuralNetwork(nn.Module):
    def __init__(self, inputDim):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(inputDim, 128),
            nn.ReLU(),
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

class training_set(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def __len__(self):
        return len(self.X)

In [36]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader)

    for batch, (X, y) in enumerate(dataloader):
        # y = y.type(torch.LongTensor)
        X, y = X.to(device), y.to(device)

        pred = model(X)
        pred = pred.float()
        y = y.unsqueeze(1)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test(dataloader, model):
    model.eval()
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            y = y.unsqueeze(1)
    mse = mean_squared_error(y, pred)
    r2 = r2_score(y, pred)
    print(f"Avg loss: {mse:>8f}, R2 Score: {r2:>8f}")
    return r2, mse

In [42]:
# df = pd.read_csv("../Data/Datasets/0.5.3-MRobust_logHenry.csv")
df = pd.read_csv("../Data/Datasets/0.5.3-MStandard_logCMC.csv")
time = dt.datetime.now().strftime("%Y-%m-%d_%H.%M.%S") #For saving files 

X = np.float32(df.iloc[:, 5:].values)
y = np.float32(df["logCMC"].values)

xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.2, random_state=42)

model = NeuralNetwork(xTrain.shape[1]).to(device)
print(model)

bat = 64

trainSet = training_set(xTrain, yTrain)
trainLoader = DataLoader(trainSet, batch_size = bat, shuffle=True)

testSet = training_set(xTest, yTest)
testLoader = DataLoader(testSet, batch_size = bat, shuffle=True)

loss_fn = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.005)

epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainLoader, model, loss_fn, optimizer)
    r2, mse = test(testLoader, model)
print(r2, mse)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=154, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
  )
)
Epoch 1
-------------------------------
loss: 21.209658 [   64/   11]
Avg loss: 2.731779, R2 Score: 0.553497
Epoch 2
-------------------------------
loss: 5.226617 [   64/   11]
Avg loss: 1.729360, R2 Score: 0.525436
Epoch 3
-------------------------------
loss: 2.676497 [   64/   11]
Avg loss: 3.008664, R2 Score: 0.471794
Epoch 4
-------------------------------
loss: 2.034390 [   64/   11]
Avg loss: 1.608071, R2 Score: 0.668545
Epoch 5
-------------------------------
loss: 1.566863 [   64/   11]
Avg loss: 3.530123, R2 Score: 0.146858
Epoch 6
-------------------------------
loss: 1.737441 [   64/   11]
Avg loss: 3.074723, R2 Score: 0.341806
Epoch 7
---------------------------

64, 32, 1: 0.7951813201817761 11.179845  
128, 32, 1: 0.9077967508296422 3.9791884  
128, 64, 1: 0.7583455947014892 8.107533  
256, 64, 1: 0.906580472756592 2.5928664

https://stackoverflow.com/questions/71998978/early-stopping-in-pytorch